# Machine learning on wine

**Topics:** Text analysis, linear regression, logistic regression, text analysis, classification

**Datasets**

- **wine-reviews.csv** Wine reviews scraped from https://www.winemag.com/
- **Data dictionary:** just go [here](https://www.winemag.com/buying-guide/tenuta-dellornellaia-2007-masseto-merlot-toscana/) and look at the page

## The background

You work in the **worst newsroom in the world**, and you've had a hard few weeks at work - a couple stories killed, a few scoops stolen out from under you. It's not going well.

And because things just can't get any worse: your boss shows up, carrying a huge binder. She slams it down on your desk.

"You know some machine learning stuff, right?"

You say "no," but she isn't listening. She's giving you an assignment, the _worst assignment_...

> Machine learning is the new maps. Let's get some hits!
>
> **Do some machine learning on this stuff.**

"This stuff" is wine reviews.

## A tiny, meagre bit of help

You have a dataset. It has some stuff in it:

* **Numbers:**
    - Year published
    - Alcohol percentage
    - Price
    - Score
    - Bottle size
* **Categories:**
    - Red vs white
    - Different countries
    - Importer
    - Designation
    - Taster
    - Variety
    - Winery
* **Free text:**
    - Wine description

# Cleaning up your data

Many of these pieces - the alcohol, the year produced, the bottle size, the country the wine is from - aren't in a format you can use. Convert the ones to numbers that are numbers, and extract the others from the appropriate strings.

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df = pd.read_csv('wine-reviews.csv')

In [3]:
df.dtypes

url                 object
wine_points        float64
wine_name           object
wine_desc           object
taster              object
price               object
designation         object
variety             object
appellation         object
winery              object
alcohol             object
bottle size         object
category            object
importer            object
date published      object
user avg rating     object
dtype: object

In [4]:
df.head(2
       )

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating
0,https://www.winemag.com/buying-guide/artadi-20...,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black pl...",Michael Schachner,"$25, Buy Now",Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5%,750 ml,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review]
1,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2012 Stoller Vineyard Chardonnay (Du...,"A tiny production wine, this is rich, tart and...",Paul Gregutt,"$65, Buy Now",Stoller Vineyard,Chardonnay,"Dundee Hills, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]


In [5]:
df['alcohol'] = df['alcohol'].str.replace('%',' ')

In [6]:
df['alcohol'] = df['alcohol'].astype(float)

In [7]:
df['price_new'] = df['price'].str.extract(r'\$(\d*)')

In [8]:
df['bottle_size_new'] = df['bottle size'].str.extract(r'(\d*)')

In [9]:
df['country'] = df['appellation'].str.extract(r', (\w*)$')


In [10]:
df['date'] = pd.to_datetime(df['date published'])

In [11]:
df = df.set_index('date')

## What might be interesting in this dataset?

Maybe start out playing around _without_ machine learning. Here are some thoughts to get you started:

* I've heard that since the 90's wine has gone through [Parkerization](https://www.estatewinebrokers.com/blog/the-parkerization-of-wine-in-the-1990s-and-beyond/), an increase in production of high-alcohol, fruity red wines thanks to the influence of wine critic Robert Parker.
* Red and white wines taste different, obviously, but people always use [goofy words to describe them](https://winefolly.com/tutorial/40-wine-descriptions/)
* Once upon a time in 1976 [California wines proved themselves against France](https://en.wikipedia.org/wiki/Judgment_of_Paris_(wine)) and France got very angry about it

In [12]:
df = df.sort_index()

In [13]:
df['2002']['wine_desc'].str.contains('floral').value_counts()

False    1265
True       19
Name: wine_desc, dtype: int64

In [14]:
df['2018']['wine_desc'].str.contains('floral').value_counts()

False    2798
True       85
Name: wine_desc, dtype: int64

In [15]:
df['2002']['wine_desc'].str.contains('fruit').value_counts()

True     717
False    567
Name: wine_desc, dtype: int64

In [16]:
df['2018']['wine_desc'].str.contains('fruit').value_counts()

True     1501
False    1382
Name: wine_desc, dtype: int64

In [17]:
df['2002']['alcohol'].mean()

13.581476323119777

In [18]:
df['2018']['alcohol'].mean()

15.925392191347168

In [19]:
df['variety'].value_counts()

Pinot Noir                                  4807
Chardonnay                                  4078
Cabernet Sauvignon                          3497
Red Blends, Red Blends                      2520
Syrah                                       1966
                                            ... 
Sirica, Italian Red                            1
Alvarinho-Chardonnay, White Blend              1
Carignan-Grenache, Rhône-style Red Blend       1
Tintilia , Italian Red                         1
Grecanico, Italian White                       1
Name: variety, Length: 500, dtype: int64

## But machine learning?

Well, you can usually break machine learning down into a few different things. These aren't necessarily perfect ways of categorizing things, but eh, close enough.

* **Predicting a number**
    - Linear regression
    - For example, how does a change in unemployment translate into a change in life expectancy?
* **Predicting a category** (aka classification)
    - Lots of algos options: logistic regression, random forest, etc
    - For example, predicting cuisines based on ingredients
* **Seeing what influences a numeric outcome**
    - Linear regression since the output is a number
    - For example, minority and poverty status on test scores 
* **Seeing what influences a categorical outcome**
    - Logistic regression since the output is a category
    - Race and car speed for if you get a waring vs ticket
    - Wet/dry pavement and car weight if you survive or not in a car crash)

We have numbers, we have categories, we have all sorts of stuff. **What are some ways we can mash them together and use machine learning?**

### Brainstorm some ideas

Use the categories above to try to come up with some ideas. Be sure to scroll up where I break down categories vs numbers vs text!

**I'll give you one idea for free:** if you don't have any ideas, start off by creating a classifier that determines whether a wine is white or red based on the wine's description.

In [78]:
df.loc[df['category']== 'White', 'type']='1'

In [79]:
df.loc[df['category']=='Red', 'type']='0'

In [80]:
#df['type'] = '0'

In [81]:
df

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating,price_new,bottle_size_new,country,type
date,,,,,,,,,,,,,,,,,,,,
2002-05-01,https://www.winemag.com/buying-guide/clos-chan...,89.0,Clos La Chance 1999 Vanumanutagi Vineyard Char...,Toasty oak and earth tones form a baseline her...,NaN,"$30, Buy Now",Vanumanutagi Vineyard,Chardonnay,"Santa Cruz Mountains, Central Coast, Californi...",Clos La Chance,14.1,750 ml,White,NaN,5/1/2002,Not rated yet [Add Your Review],30,750,US,1
2002-05-01,https://www.winemag.com/buying-guide/schug-200...,87.0,Schug 2000 Heritage Reserve Chardonnay (Carneros),Richer than this winery's other two current re...,NaN,"$30, Buy Now",Heritage Reserve,Chardonnay,"Carneros, Napa-Sonoma, California, US",Schug,NaN,750 ml,White,NaN,5/1/2002,Not rated yet [Add Your Review],30,750,US,1
2002-05-01,https://www.winemag.com/buying-guide/colosi-19...,87.0,Colosi 1999 Nero d'Avola (Sicilia),Six months of aging in Slavonian oak has impar...,Joe Czerwinski,"$12, Buy Now",NaN,"Nero d'Avola, Italian Red","Sicilia, Sicily & Sardinia, Italy",Colosi,12.0,750 ml,Red,Vias Imports,5/1/2002,Not rated yet [Add Your Review],12,750,Italy,0
2002-05-01,https://www.winemag.com/buying-guide/bodega-ca...,87.0,Bodega Catena Zapata 1999 Catena Agrelo Cabern...,"Made in a woody style, but there's enough swee...",Joe Czerwinski,"$20, Buy Now",Catena Agrelo,Cabernet Sauvignon,"Mendoza, Mendoza Province, Argentina",Bodega Catena Zapata,13.6,750 ml,Red,Billington Imports,5/1/2002,Not rated yet [Add Your Review],20,750,Argentina,0
2002-05-01,https://www.winemag.com/buying-guide/covey-run...,87.0,Covey Run 1999 Chardonnay (Columbia Valley (WA)),"Starting with a creamy, toasty, tropical-fruit...",NaN,"$12, Buy Now",NaN,Chardonnay,"Columbia Valley (WA), Columbia Valley, Washing...",Covey Run,13.5,750 ml,White,NaN,5/1/2002,Not rated yet [Add Your Review],12,750,US,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31,https://www.winemag.com/buying-guide/small-vin...,92.0,Small Vines 2016 Estate Cuvée Pinot Noir (Russ...,"Fresh and forested, this wine has tannin weigh...",Virginie Boone,"$65, Buy Now",Estate Cuvée,Pinot Noir,"Russian River Valley, Sonoma, California, US",Small Vines,13.5,750 ml,Red,NaN,12/31/2018,Not rated yet [Add Your Review],65,750,US,0
2018-12-31,https://www.winemag.com/buying-guide/ryan-coch...,93.0,Ryan Cochrane 2016 Fiddlestix Vineyard Pinot N...,"Bright and energetic aromas of pomegranate, li...",Matt Kettmann,"$46, Buy Now",Fiddlestix Vineyard,Pinot Noir,"Sta. Rita Hills, Central Coast, California, US",Ryan Cochrane,14.3,750 ml,Red,NaN,12/31/2018,Not rated yet [Add Your Review],46,750,US,0
2018-12-31,https://www.winemag.com/buying-guide/salvaterr...,93.0,SalvaTerra 2009 Cave di Prun Riserva (Amarone...,"Enticing aromas of ripe dark-skinned berry, ne...",Kerin O’Keefe,"$75, Buy Now",Cave di Prun Riserva,"Red Blends, Red Blends","Amarone della Valpolicella Classico, Veneto, I...",SalvaTerra,15.5,750 ml,Red,Salvaterra USA,12/31/2018,Not rated yet [Add Your Review],75,750,Italy,0


In [82]:
df['category'].value_counts()

Red            25392
White          11922
Sparkling       2519
Rose            1329
Dessert          744
Port/Sherry      358
Fortified         31
Name: category, dtype: int64

In [83]:
train_df = df.fillna('0')

In [84]:
df.head(5)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating,price_new,bottle_size_new,country,type
date,,,,,,,,,,,,,,,,,,,,
2002-05-01,https://www.winemag.com/buying-guide/clos-chan...,89.0,Clos La Chance 1999 Vanumanutagi Vineyard Char...,Toasty oak and earth tones form a baseline her...,NaN,"$30, Buy Now",Vanumanutagi Vineyard,Chardonnay,"Santa Cruz Mountains, Central Coast, Californi...",Clos La Chance,14.1,750 ml,White,NaN,5/1/2002,Not rated yet [Add Your Review],30,750,US,1
2002-05-01,https://www.winemag.com/buying-guide/schug-200...,87.0,Schug 2000 Heritage Reserve Chardonnay (Carneros),Richer than this winery's other two current re...,NaN,"$30, Buy Now",Heritage Reserve,Chardonnay,"Carneros, Napa-Sonoma, California, US",Schug,NaN,750 ml,White,NaN,5/1/2002,Not rated yet [Add Your Review],30,750,US,1
2002-05-01,https://www.winemag.com/buying-guide/colosi-19...,87.0,Colosi 1999 Nero d'Avola (Sicilia),Six months of aging in Slavonian oak has impar...,Joe Czerwinski,"$12, Buy Now",NaN,"Nero d'Avola, Italian Red","Sicilia, Sicily & Sardinia, Italy",Colosi,12.0,750 ml,Red,Vias Imports,5/1/2002,Not rated yet [Add Your Review],12,750,Italy,0
2002-05-01,https://www.winemag.com/buying-guide/bodega-ca...,87.0,Bodega Catena Zapata 1999 Catena Agrelo Cabern...,"Made in a woody style, but there's enough swee...",Joe Czerwinski,"$20, Buy Now",Catena Agrelo,Cabernet Sauvignon,"Mendoza, Mendoza Province, Argentina",Bodega Catena Zapata,13.6,750 ml,Red,Billington Imports,5/1/2002,Not rated yet [Add Your Review],20,750,Argentina,0
2002-05-01,https://www.winemag.com/buying-guide/covey-run...,87.0,Covey Run 1999 Chardonnay (Columbia Valley (WA)),"Starting with a creamy, toasty, tropical-fruit...",NaN,"$12, Buy Now",NaN,Chardonnay,"Columbia Valley (WA), Columbia Valley, Washing...",Covey Run,13.5,750 ml,White,NaN,5/1/2002,Not rated yet [Add Your Review],12,750,US,1


In [92]:
train_df = pd.DataFrame({
    'type': df.type,
    'floral': df.wine_desc.str.contains("floral", na=False).astype(int), 
    'fruity': df.wine_desc.str.contains("fruit", na=False).astype(int),
    'oak': df.wine_desc.str.contains("oak", na=False).astype(int),
    'nutty': df.wine_desc.str.contains("nut", na=False).astype(int),
    'dry': df.wine_desc.str.contains("dry", na=False).astype(int),
    'smooth': df.wine_desc.str.contains("smooth", na=False).astype(int),
    'sweet': df.wine_desc.str.contains("sweet", na=False).astype(int),
    'rich': df.wine_desc.str.contains("rich", na=False).astype(int),
    'moody': df.wine_desc.str.contains("mood", na=False).astype(int),
    
    
})
train_df



,type,floral,fruity,oak,nutty,dry,smooth,sweet,rich,moody
date,,,,,,,,,,
2002-05-01,1,0,0,1,0,0,0,0,0,0
2002-05-01,1,0,1,1,0,0,0,1,0,0
2002-05-01,0,0,1,1,0,0,0,0,0,0
2002-05-01,0,0,1,1,0,0,0,1,0,0
2002-05-01,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2018-12-31,0,0,1,1,0,0,0,0,0,0
2018-12-31,0,0,0,0,0,0,0,0,0,0
2018-12-31,0,0,0,0,0,0,0,0,0,0


In [93]:
X = train_df.drop(columns=['type'])
y = train_df.type

In [94]:
X.head(5)

,floral,fruity,oak,nutty,dry,smooth,sweet,rich,moody
date,,,,,,,,,
2002-05-01,0,0,1,0,0,0,0,0,0
2002-05-01,0,1,1,0,0,0,1,0,0
2002-05-01,0,1,1,0,0,0,0,0,0
2002-05-01,0,1,1,0,0,0,1,0,0
2002-05-01,0,1,0,1,0,1,0,0,0


In [95]:
y.head(5)

date
2002-05-01    1
2002-05-01    1
2002-05-01    0
2002-05-01    0
2002-05-01    1
Name: type, dtype: object

In [96]:
train_df.isnull().values.any()

False

In [97]:
from sklearn.linear_model import LogisticRegression
X = train_df.drop(columns='type')
y = train_df.type
clf = LogisticRegression(C=1e9, solver='lbfgs')

clf.fit(X, y)

LogisticRegression(C=1000000000.0)

In [91]:
clf.score(X,y)

0.7180281357134413

In [99]:
from sklearn.metrics import confusion_matrix
y_true = y
y_pred = clf.predict(X)


matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['white', 'red'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted white,Predicted red
Is white,30330,43
Is red,11878,44


You can also go to https://library.columbia.edu and see if you can find some academic papers about wine. I'm sure they'll inspire you! (and they might even have some ML ideas in them you can steal, too)

# Implement 2 of your machine learning ideas

In [102]:
df.head(1)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating,price_new,bottle_size_new,country,type
date,,,,,,,,,,,,,,,,,,,,
2002-05-01,https://www.winemag.com/buying-guide/clos-chan...,89.0,Clos La Chance 1999 Vanumanutagi Vineyard Char...,Toasty oak and earth tones form a baseline her...,NaN,"$30, Buy Now",Vanumanutagi Vineyard,Chardonnay,"Santa Cruz Mountains, Central Coast, Californi...",Clos La Chance,14.1,750 ml,White,NaN,5/1/2002,Not rated yet [Add Your Review],30,750,US,1


In [106]:
import statsmodels.formula.api as smf

model = smf.ols('wine_points ~ price_new', data=df)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            wine_points   R-squared:                       0.403
Model:                            OLS   Adj. R-squared:                  0.398
Method:                 Least Squares   F-statistic:                     88.01
Date:                Thu, 01 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:57:59   Log-Likelihood:                -93620.
No. Observations:               39301   AIC:                         1.878e+05
Df Residuals:                   39001   BIC:                         1.904e+05
Df Model:                         299                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            85.0658      0.078   1091.565      0.000      84.913      85.219
price_new[T.100]      7.3822      0.184     40.188      0.000       7.022       7.742
price_new[T.1000]    10.9342      1.861      5.874      0.000       7.286      14.582
price_new[T.101]      7.9342      2.631      3.015      0.003       2.777      13.091
price_new[T.102]      6.6342      0.835      7.942      0.000       4.997       8.271
price_new[T.103]      6.6842      0.933      7.163      0.000       4.855       8.513
price_new[T.104]      5.9342      0.997      5.951      0.000       3.980       7.889
price_new[T.105]      7.4342      0.423     17.571      0.000       6.605       8.263
price_new[T.106]      6.4897      0.880      7.373      0.000       4.765       8.215
price_new[T.107]      5.6008      1.520      3.684      0.000       2.621       8.581
price_new[T.108]      8.1649      0.734     11.130      0.000       6.727       9.603
price_new[T.109]      7.2199      0.997      7.241      0.000       5.265       9.174
price_new[T.11]       0.4148      0.128      3.242      0.001       0.164       0.666
price_new[T.110]      6.7278      0.340     19.764      0.000       6.061       7.395
price_new[T.111]      8.6842      1.317      6.592      0.000       6.102      11.266
price_new[T.112]      7.2199      0.997      7.241      0.000       5.265       9.174
price_new[T.1125]     8.9342      2.631      3.395      0.001       3.777      14.091
price_new[T.113]      7.9342      1.179      6.731      0.000       5.624      10.245
price_new[T.114]      7.2675      1.077      6.751      0.000       5.157       9.378
price_new[T.115]      7.8887      0.404     19.522      0.000       7.097       8.681
price_new[T.116]      6.6008      1.520      4.341      0.000       3.621       9.581
price_new[T.117]      7.5342      1.179      6.391      0.000       5.224       9.845
price_new[T.118]      7.6484      0.997      7.671      0.000       5.694       9.603
price_new[T.119]      7.4342      1.077      6.906      0.000       5.324       9.544
price_new[T.12]       0.5547      0.109      5.075      0.000       0.340       0.769
price_new[T.120]      7.8417      0.253     30.948      0.000       7.345       8.338
price_new[T.1200]    11.4342      1.861      6.143      0.000       7.786      15.082
price_new[T.121]      8.2675      1.520      5.437      0.000       5.287      11.248
price_new[T.122]      7.9342      2.631      3.015      0.003       2.777      13.091
price_new[T.123]      7.6008      1.520      4.999      0.000       4.621      10.581
price_new[T.124]      6.9342      2.631      2.635      0.008       1.777      12.091
price_new[T.125]      7.9429      0.258     30.743      0.000       7.437       8.449
price_new[T.126]      5.6008      1.077      5.203      0.000       3.491